# Terminal.py 功能测试

本notebook用于测试terminal.py中的核心功能，包括：
1. WebSocket连接建立
2. 终端会话创建
3. 数据发送和接收
4. 心跳机制
5. 终端大小调整

In [13]:
import websockets
import asyncio
import json
import time
from contextlib import asynccontextmanager

@asynccontextmanager
async def terminal_connection():
    """创建到terminal WebSocket的连接"""
    uri = "ws://localhost:8007/ws/terminal"
    async with websockets.connect(uri) as websocket:
        yield websocket

async def test_terminal_basic_functionality():
    """测试终端基本功能"""
    async with terminal_connection() as ws:
        # 测试1: 验证连接建立
        print("1. 测试连接建立...")
        initial_response = await ws.recv()
        print(f"收到初始响应: {initial_response}")        
        
        # 测试2: 发送终端调整大小命令
        print("\n2. 测试终端大小调整...")
        resize_data = {
            "type": "resize",
            "cols": 80,
            "rows": 24
        }
        await ws.send(json.dumps(resize_data))

        # 等待并收集输出
        response = ""
        start_time = time.time()
        while time.time() - start_time < 2:  # 等待2秒
            try:
                msg = await asyncio.wait_for(ws.recv(), timeout=0.5)
                response += msg if isinstance(msg, str) else msg.decode('utf-8')
                print(response)
            except asyncio.TimeoutError:
                continue
        
        # 测试3: 发送简单命令并验证响应
        print("\n3. 测试命令执行...")
        await ws.send("echo 'Hello Terminal'\n")
        
        # 等待并收集输出
        response = ""
        start_time = time.time()
        while time.time() - start_time < 10:  # 等待2秒
            try:
                msg = await asyncio.wait_for(ws.recv(), timeout=0.5)
                response += msg if isinstance(msg, str) else msg.decode('utf-8')
                print(response)
                if 'Hello Terminal' in response:
                    break
            except asyncio.TimeoutError:
                continue
                
        print(f"命令执行响应: {response}")
        assert 'Hello Terminal' in response                        
        
        print("\n所有测试通过！")

# 运行测试
await test_terminal_basic_functionality()

1. 测试连接建立...
收到初始响应: 
The default interactive shell is now zsh.
To update your account to use zsh, please run `chsh -s /bin/zsh`.
For more details, please visit https://support.apple.com/kb/HT208050.


2. 测试终端大小调整...
]633;A(byzerllm) (byzerllm) [w@me auto-coder.web]$ ]633;B
]633;A(byzerllm) (byzerllm) [w@me auto-coder.web]$ ]633;B33;B

3. 测试命令执行...
echo 'Hello Terminal'
Hello Terminal
]633;A(byzerllm) (byzerllm) [w@me auto-coder.web]$ ]633;B
命令执行响应: echo 'Hello Terminal'
Hello Terminal
]633;A(byzerllm) (byzerllm) [w@me auto-coder.web]$ ]633;B

所有测试通过！


## 压力测试

测试多个并发连接的情况

In [ ]:
async def stress_test(num_connections=5):
    """创建多个并发连接进行压力测试"""
    print(f"开始{num_connections}个并发连接的压力测试...")
    
    async def single_connection_test(conn_id):
        try:
            async with terminal_connection() as ws:
                # 发送一些基本命令
                await ws.send(f"echo 'Connection {conn_id}'\n")
                response = await ws.recv()
                print(f"连接 {conn_id} 收到响应: {response}")
                
                # 保持连接一段时间
                await asyncio.sleep(2)
                
                return True
        except Exception as e:
            print(f"连接 {conn_id} 发生错误: {str(e)}")
            return False
    
    # 创建多个并发连接
    tasks = [single_connection_test(i) for i in range(num_connections)]
    results = await asyncio.gather(*tasks)
    
    # 统计结果
    success_count = sum(1 for r in results if r)
    print(f"\n压力测试完成:")
    print(f"成功连接: {success_count}/{num_connections}")
    
# 运行压力测试
await stress_test()